# import dataset

In [3]:
file_path = 'C:/Users/Eduardo/OneDrive/Documents/Masters python/Machine learning/spotify_listenership_sample (1).csv'

# Data summary

In [4]:
import pandas as pd
# import pandas to look at data


#read file 
data = pd.read_csv(file_path)

# let's look at the first few rows of the dataset
print(data.head())

                 track_id            track_name          track_artist  \
0  1iTZnsVqhCBdZqp9gjITDB            Run To You               Code 64   
1  42LiXQn3xibOEodtBMoJp3  Drop The Bomb On 'Em                Eminem   
2  2cTGafydY9vdnqJQ2Gw6t8           Butterflies                  Myon   
3  2boJnT3S2aSBagFEUTXrfx      Get Your Roll On            Big Tymers   
4  5HzekjQF7xLqBVeynqprDk          By Your Side  Ballin Entertainment   

  track_album_release_date  track_duration_ms  sound_danceability  \
0                6/16/2006             267747               0.445   
1                5/15/2009             287267               0.892   
2                6/22/2018             234375               0.539   
3                 1/1/2007             245200               0.849   
4                4/20/2008             198165               0.802   

   sound_energy  sound_key  sound_loudness  sound_mode  ...  sound_liveness  \
0         0.999          0          -5.627           1  ...        

non-predictive variables:
- track_id
- track_name
- track_artist
- track_album_release_date

Binary variables
- genre_edm
- genre_latin
- genre_pop
- genre_rnb
- genre_rap
- genre_rock
- has_leistenership

In [5]:
# Let's look If the variables have any Null Values and what data type is it
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  2999 non-null   object 
 1   track_name                2999 non-null   object 
 2   track_artist              2999 non-null   object 
 3   track_album_release_date  2999 non-null   object 
 4   track_duration_ms         2999 non-null   int64  
 5   sound_danceability        2999 non-null   float64
 6   sound_energy              2999 non-null   float64
 7   sound_key                 2999 non-null   int64  
 8   sound_loudness            2999 non-null   float64
 9   sound_mode                2999 non-null   int64  
 10  sound_speechiness         2999 non-null   float64
 11  sound_acousticness        2999 non-null   float64
 12  sound_instrumentalness    2999 non-null   float64
 13  sound_liveness            2999 non-null   float64
 14  sound_va

# Prepare data

In [6]:
from sklearn.preprocessing import StandardScaler
# We use Standard Scaler before feeding it into the model

#Normalize numerical features
# select numerical features you want to normalize (exlude the binary variables and the non-predictive features).  

numerical_features = data[['track_duration_ms', 'sound_danceability', 'sound_energy', 'sound_key',
                          'sound_loudness', 'sound_speechiness', 'sound_acousticness',
                          'sound_instrumentalness', 'sound_liveness', 
                          'sound_instrumentalness', 'sound_liveness','sound_valence', 'sound_tempo']]
# Create the StandardScaler
scaler = StandardScaler()

#scale the numerical features
data[numerical_features.columns] = scaler.fit_transform(numerical_features)

prepare for modeling

In [22]:
#split the data
from sklearn.model_selection import train_test_split

# Separate features from the target variable

#Drop the non-predictive columns 
x = data.drop(['track_id', 'track_name', 'track_artist', 'track_album_release_date', 'has_listenership'], axis = 1)

# Define Target variable
y= data['has_listenership']

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# confirm the split
print("Training set dimensions:", x_train.shape, y_train.shape)
print("Testing set dimensions:", x_test.shape, y_test.shape)

Training set dimensions: (2399, 18) (2399,)
Testing set dimensions: (600, 18) (600,)


# Data modeling

SVM model

In [17]:
# Import then necessary library for SVM
from sklearn.svm import SVC

# Create an SVM classifier
svm_model = SVC(kernel = 'rbf', C=1.0, gamma ='auto', random_state = 42) 

# Fit the model in the training data 
svm_model.fit(x_train, y_train)

#Predict on the training and testing data
svm_train_predictions = svm_model.predict(x_train)
svm_test_predictions = svm_model.predict(x_test)

In [12]:
# Evaluation metrics

# Import the necessary libraries 
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Evaluate the model's performance on the training set
print("SVM Training Performance:")
print(classification_report(y_train, svm_train_predictions))
print("Training Accuracy:", accuracy_score(y_train, svm_train_predictions))

# Evaluate the model's performance on the testing set
print("\nSVM Testing Performance:")
print(classification_report(y_test, svm_test_predictions))
print("Testing Accuracy:", accuracy_score(y_test, svm_test_predictions))

# Output the confusion matrix for the testing set
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, svm_test_predictions))

SVM Training Performance:
              precision    recall  f1-score   support

           0       0.69      0.62      0.65      1184
           1       0.66      0.73      0.70      1215

    accuracy                           0.68      2399
   macro avg       0.68      0.68      0.68      2399
weighted avg       0.68      0.68      0.68      2399

Training Accuracy: 0.6765318882867861

SVM Testing Performance:
              precision    recall  f1-score   support

           0       0.64      0.53      0.58       315
           1       0.56      0.67      0.61       285

    accuracy                           0.60       600
   macro avg       0.60      0.60      0.60       600
weighted avg       0.60      0.60      0.60       600

Testing Accuracy: 0.5966666666666667

Confusion Matrix:
[[168 147]
 [ 95 190]]


The test are 67% and 59% accurate

# textual features

Bag-of-words

In [9]:
# import necessary library
from sklearn.feature_extraction.text import CountVectorizer

#create the vectorizer
vectorizer = CountVectorizer()

# Fit and transform 'track_name' to create the Bag-of-Words feature matrix
X_bow = vectorizer.fit_transform(data['track_name'])

# Convert to DataFrame (optional, for better integration with pandas)
X_bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

Tf - IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the tfidVfectorizer
tfidf = TfidfVectorizer()

# Fit and transform 'track_name' to create the TF-IDF feature matrix
x_tfidf = tfidf.fit_transform(data['track_name'])

In [12]:
# Convert the tfidf_matrix, which is a sparse matrix, to a DataFrame
tfidf_features = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Combine the numerical features with the TF-IDF features
# Ensure that indices are aligned between the dataframes
combined_features = pd.concat([numerical_features.reset_index(drop=True), tfidf_features.reset_index(drop=True)], axis=1)

# Combine datasets

In [25]:
# Combine numerical features with BoW features
X_combined_bow = pd.concat([numerical_features.reset_index(drop=True), X_bow_df.reset_index(drop=True)], axis=1)

# Combine numerical features with TF-IDF features
X_combined_tfidf = pd.concat([numerical_features.reset_index(drop=True), tfidf_features.reset_index(drop=True)], axis=1)

# Define the target variable (assuming it's a binary classification problem)
y = data['has_listenership']  

In [26]:
# Split data for BoW features
X_train_bow, X_test_bow, y_train, y_test = train_test_split(X_combined_bow, y, test_size=0.2, random_state=42)

# Split data for TF-IDF features
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_combined_tfidf, y, test_size=0.2, random_state=42)

# Modeling with textual features

SVM model again (with bag of words)

In [34]:
from sklearn.metrics import confusion_matrix
# Initialize the SVM model with RBF kernel
svm_bow = SVC(kernel='rbf', C=1.0, gamma='auto', random_state=42)

# Fit the model on the training data
svm_bow.fit(X_train_bow, y_train)

# Predict on the training and testing data
svm_train_predictions_bow = svm_bow.predict(X_train_bow)
svm_test_predictions_bow = svm_bow.predict(X_test_bow)

# Evaluate the model's performance on the training set
print("SVM Training Performance (BoW):")
print(classification_report(y_train, svm_train_predictions_bow))
print("Training Accuracy:", accuracy_score(y_train, svm_train_predictions_bow))

# Evaluate the model's performance on the testing set
print("\nSVM Testing Performance (BoW):")
print(classification_report(y_test, svm_test_predictions_bow))
print("Testing Accuracy:", accuracy_score(y_test, svm_test_predictions_bow))

# Output the confusion matrix for the testing set
print("\nConfusion Matrix (BoW):")
print(confusion_matrix(y_test, svm_test_predictions_bow))


SVM Training Performance (BoW):
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1184
           1       0.93      0.94      0.94      1215

    accuracy                           0.94      2399
   macro avg       0.94      0.94      0.94      2399
weighted avg       0.94      0.94      0.94      2399

Training Accuracy: 0.9353897457273864

SVM Testing Performance (BoW):
              precision    recall  f1-score   support

           0       0.57      0.64      0.60       315
           1       0.54      0.47      0.51       285

    accuracy                           0.56       600
   macro avg       0.56      0.56      0.55       600
weighted avg       0.56      0.56      0.56       600

Testing Accuracy: 0.56

Confusion Matrix (BoW):
[[201 114]
 [150 135]]


SVM model (with tf-idf)

In [30]:
# Initialize the SVM model with RBF kernel
svm_tfidf = SVC(kernel='rbf', C=1.0, gamma='auto', random_state=42)

# Fit the model on the training data
svm_tfidf.fit(X_train_tfidf, y_train)

# Predict on the training and testing data
svm_train_predictions_tfidf = svm_tfidf.predict(X_train_tfidf)
svm_test_predictions_tfidf = svm_tfidf.predict(X_test_tfidf)

# Evaluate the model's performance on the training set
print("SVM Training Performance (TF-IDF):")
print(classification_report(y_train, svm_train_predictions_tfidf))
print("Training Accuracy:", accuracy_score(y_train, svm_train_predictions_tfidf))

# Evaluate the model's performance on the testing set
print("\nSVM Testing Performance (TF-IDF):")
print(classification_report(y_test, svm_test_predictions_tfidf))
print("Testing Accuracy:", accuracy_score(y_test, svm_test_predictions_tfidf))

# Output the confusion matrix for the testing set
print("\nConfusion Matrix (TF-IDF):")
print(confusion_matrix(y_test, svm_test_predictions_tfidf))


SVM Training Performance (TF-IDF):
              precision    recall  f1-score   support

           0       0.94      0.93      0.93      1184
           1       0.93      0.94      0.94      1215

    accuracy                           0.93      2399
   macro avg       0.94      0.93      0.93      2399
weighted avg       0.93      0.93      0.93      2399

Training Accuracy: 0.9349729053772405

SVM Testing Performance (TF-IDF):
              precision    recall  f1-score   support

           0       0.57      0.64      0.60       315
           1       0.54      0.47      0.51       285

    accuracy                           0.56       600
   macro avg       0.56      0.56      0.55       600
weighted avg       0.56      0.56      0.56       600

Testing Accuracy: 0.56

Confusion Matrix (TF-IDF):
[[201 114]
 [150 135]]


Logistic regression (with bag-of-words)

In [33]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Initialize the Logistic Regression model
log_reg_bow = LogisticRegression(max_iter=1000, random_state=42)

# Fit the model on the training data
log_reg_bow.fit(X_train_bow, y_train)

# Predict on the training and testing data
log_reg_train_predictions_bow = log_reg_bow.predict(X_train_bow)
log_reg_test_predictions_bow = log_reg_bow.predict(X_test_bow)

# Evaluate the model's performance on the training set
print("Logistic Regression Training Performance (BoW):")
print(classification_report(y_train, log_reg_train_predictions_bow, zero_division=0))
print("Training Accuracy:", accuracy_score(y_train, log_reg_train_predictions_bow))

# Evaluate the model's performance on the testing set
print("\nLogistic Regression Testing Performance (BoW):")
print(classification_report(y_test, log_reg_test_predictions_bow, zero_division=0))
print("Testing Accuracy:", accuracy_score(y_test, log_reg_test_predictions_bow))

# Output the confusion matrix for the testing set
print("\nConfusion Matrix (BoW):")
print(confusion_matrix(y_test, log_reg_test_predictions_bow))


Logistic Regression Training Performance (BoW):
              precision    recall  f1-score   support

           0       0.49      1.00      0.66      1184
           1       0.00      0.00      0.00      1215

    accuracy                           0.49      2399
   macro avg       0.25      0.50      0.33      2399
weighted avg       0.24      0.49      0.33      2399

Training Accuracy: 0.49353897457273865

Logistic Regression Testing Performance (BoW):
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       315
           1       0.00      0.00      0.00       285

    accuracy                           0.53       600
   macro avg       0.26      0.50      0.34       600
weighted avg       0.28      0.53      0.36       600

Testing Accuracy: 0.525

Confusion Matrix (BoW):
[[315   0]
 [285   0]]


Logistic regression (with TF-Idf)

In [38]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
log_reg_tfidf = LogisticRegression(max_iter=1000, random_state=42)

# Fit the model on the training data
log_reg_tfidf.fit(X_train_tfidf, y_train)

# Predict on the training and testing data
log_reg_train_predictions_tfidf = log_reg_tfidf.predict(X_train_tfidf)
log_reg_test_predictions_tfidf = log_reg_tfidf.predict(X_test_tfidf)

# Evaluate the model's performance on the training set
print("Logistic Regression Training Performance (TF-IDF):")
print(classification_report(y_train, log_reg_train_predictions_tfidf, zero_division=0))
print("Training Accuracy:", accuracy_score(y_train, log_reg_train_predictions_tfidf))

# Evaluate the model's performance on the testing set
print("\nLogistic Regression Testing Performance (TF-IDF):")
print(classification_report(y_test, log_reg_test_predictions_tfidf, zero_division=0))
print("Testing Accuracy:", accuracy_score(y_test, log_reg_test_predictions_tfidf))

# Output the confusion matrix for the testing set
print("\nConfusion Matrix (TF-IDF):")
print(confusion_matrix(y_test, log_reg_test_predictions_tfidf))


Logistic Regression Training Performance (TF-IDF):
              precision    recall  f1-score   support

           0       0.49      1.00      0.66      1184
           1       0.00      0.00      0.00      1215

    accuracy                           0.49      2399
   macro avg       0.25      0.50      0.33      2399
weighted avg       0.24      0.49      0.33      2399

Training Accuracy: 0.49353897457273865

Logistic Regression Testing Performance (TF-IDF):
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       315
           1       0.00      0.00      0.00       285

    accuracy                           0.53       600
   macro avg       0.26      0.50      0.34       600
weighted avg       0.28      0.53      0.36       600

Testing Accuracy: 0.525

Confusion Matrix (TF-IDF):
[[315   0]
 [285   0]]


# Topic Modeling (LDA) 

In [40]:
from sklearn.decomposition import LatentDirichletAllocation
 #Step 1: Use CountVectorizer to transform the song titles into a matrix of token counts
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(data['track_name'])  # Document-term matrix

# Step 2: Fit LDA to the document-term matrix
n_components = 5  
lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
lda.fit(dtm)

# Step 3: Print the top words in each topic
words = cv.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx}:")
    print(" ".join([words[i] for i in topic.argsort()[-10:]]))

Topic #0:
stay paradise album time yo el live remix remastered version
Topic #1:
dance extended sweet te life light night original mix remix
Topic #2:
big make want da got girl way radio edit feat
Topic #3:
forever tú ya black lost man day world feat good
Topic #4:
gonna feel wanna remaster don let remix la like love


Identify Thematic modeling:

Topic #0: This topic seems to include songs that are potentially about nostalgia or enduring classics, as suggested by words like "stay," "paradise," and "remastered version." These could be songs that have been remixed or re-released.

Topic #1: The words "dance," "extended," "night," and "remix" indicate this topic might be related to dance music or club tracks, with a focus on extended mixes and remixes that are commonly played in nightlife settings.

Topic #2: With words like "big," "make," "want," "radio edit," and "feat" (featuring), this topic might encompass mainstream, radio-friendly music that often features collaborations between artists.

Topic #3: The presence of words like "forever," "lost," "world," and "good" suggests themes of love, loss, and perhaps introspection. The word "feat" again indicates collaborations, and "black" could refer to either a mood or be part of a band name or song title.

Topic #4: This topic seems to focus on emotive content, with words like "gonna," "feel," "wanna," "love," and "la" (which could refer to 'the' in Spanish or shorthand for Los Angeles). The songs might be about personal feelings, desires, and relationships